In [ ]:
import sys, os, json, random, pickle
sys.path.append('../app')

import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import auc, roc_auc_score, roc_curve, accuracy_score, precision_score, classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV

import lightgbm as lgb

from policy import HighLevelPolicy
from virtual_game import VirtualGame

In [ ]:
def create_data():
    policy = Policy(load_states=True)
    
    config = Config()
    config.blue_team_size = 4
    config.red_team_size = 4
    
    q_state_actions = get_state_actions(policy)
    print('%d state actions:\n%s' % (len(state_actions), '\n'.join(list(state_actions.items())[:5])))
    
    for i in range(100):
        
        q, x, desc = get_random_policy(q_state_actions, policy)

        policy.q = q
        
        game = VirtualGame(config)
        
        for i in range(steps):
            for player_map_idx in range(len(game.the_map.players)):
                action = policy.get_high_level_action(player_map_idx, the_map)
                
                reward, winner = game.step(player_map_idx, action)
                if winner:
                    
    
    
def get_state_actions(policy):
    state_actions = {}
    
    for state_code in policy.high_level_state_codes:
        hls_str = get_hls(state_code, policy)
        state_actions[hls_str]=[]
        
        for i,act in enumerate(policy.high_level_actions):
            if 'self_incapacitated' in hls and not act=='wait':
                continue
            if act=='go_opponent_flag_carrier' and not 'team_flag_in_play' in hls:
                continue
            elif act=='gaurd_teammate_flag_carrier':
                if 'self_has_flag' in hls or not 'opponents_flag_in_play' in hls:
                    continue
                    
            state_actions[hls_str].append(i)
            
    return state_actions


def get_hls(state_code, policy):
    hls = []
    for i,s in enumerate(policy.high_level_states):
        if state_code[i]:
            hls.append(s)
    hls_str = ', '.join(hls) if hls else 'none'
    return hls_str


def get_random_policy(state_actions, policy):
    policy_desc = []
    x = []
    q = np.zeros_like(policy.q)
    
    for state_idx, state_code in enumerate(policy.high_level_state_codes):
        hls_str = get_hls(state_code, policy)
        action_idx = random.choice(state_actions[hls_str])
        q[state_idx, action_idx] = 1.0
        policy_desc.append('%s : %s' % (hls_str, policy.high_level_actions[action_idx]))
        x.append(action_idx)
        
    return q, np.array(x), policy_desc

In [ ]:
x_train, y_train, x_test, y_test, feat_names

In [ ]:
def train_test_lgb(x_train, y_train, x_test, y_test, feats=[]):
    scaler = StandardScaler()
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.transform(x_test)
    
    lr, n_estimators = get_best_lr_nestimators(x_train, y_train, x_test, y_test)
    
    gbm = lgb.LGBMClassifier(
        learning_rate = lr, 
        metric = 'l1', 
        n_estimators = n_estimators)

    gbm.fit(x_train, y_train,
            eval_set=[(x_test, y_test)],
            eval_metric=['auc', 'binary_logloss'], early_stopping_rounds=5)
    
    report_results(gbm, x_test, y_test, feats)
    
    return gbm
    
    
def get_best_lr_nestimators(x_train, y_train, x_test, y_test):
    estimator = lgb.LGBMClassifier(
        learning_rate = 0.125, 
        metric = 'l1', 
        n_estimators = 20,
        num_leaves = 38)

    param_grid = {
        'n_estimators': [x for x in range(24,40,2)],
        'learning_rate': [0.10, 0.125, 0.15, 0.175, 0.2, 0.225, 0.25]}
    
    gridsearch = GridSearchCV(estimator, param_grid)

    gridsearch.fit(x_train, y_train,
            eval_set = [(x_test, y_test)],
            eval_metric = ['auc', 'binary_logloss'],
            early_stopping_rounds = 5)
    
    return gridsearch.best_params_['learning_rate'], gridsearch.best_params_['n_estimators']
    

def report_results(gbm, x_test, y_test, feats):
    if feats:
        print('Feature Importance:')
        for i in reversed(np.argsort(gbm.feature_importances_).tolist()):
            print('\t%s\t%d' % (feats[i], gbm.feature_importances_[i]))
        
    y_pred = gbm.predict(x_test, num_iteration=gbm.best_iteration_)
    y_pred_prob = gbm.predict_proba(x_test)[:, 1]
    
    print('\n\n', str(classification_report(y_test, y_pred, digits=4)))
    print('\n\nTN FP\nFN TP\n', confusion_matrix(y_test, y_pred))
    
    print('\n\nAUC ROC: ', str(roc_auc_score(y_test, y_pred_prob)))
    

def save_model(lgbm_model, path='lgbm.model'):
    lgbm_model.booster_.save_model(path)
    
    
def load_model(path='lgbm.model'):
    return lgb.Booster(model_file=path)

In [ ]:
lgbm_model = train_test_lgb(x_train, y_train, x_test, y_test, feat_names)
save_model(lgbm_model, 'lgbm_arr_syn_10000.model')

In [ ]:
report_results(gbm, x_test, y_test, feat_names)

In [ ]:
def logistic_regression(x_train, y_train, x_test, y_test, feat_names, data, an_pairs_test):
    scaler = StandardScaler()
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.transform(x_test)
    
    clf = LogisticRegression().fit(x_train, y_train)
    
    print('Feature coefficients:')
    wts = clf.coef_.squeeze()
    for i,feat in enumerate(feat_names):
        print(wts[i], feat)

    preds = clf.predict(x_test)
    probs = clf.predict_proba(x_test)

    cor=0
    nm = 'logistic_regression_error_records.txt'
    with open(nm, 'w', encoding='utf-8') as o:
        for i in range(preds.shape[0]):
            trg = y_test[i]
            pred = preds[i]
            prob = probs[i][pred]
            src_rec, trg_rec, _, _ = data[i]
            src_an, trg_an, details = an_pairs_test[i]
            
            if trg==pred:
                cor+=1
            else:
                o.write('Target: %d, Pred: %d, Prob: %.4f\nSource:\n%s\n\nTarget:\n%s\n\nSim Details:\n%s\n\n\n' % (trg, 
                         pred, prob, str(src_rec), str(trg_rec), str(details)))
    print(cor/preds.shape[0])
    return clf

In [ ]:
logistic_regression(x_train, y_train, x_test, y_test, feat_names, data, an_pairs_test)